### Write benchmark commands into a file benchmark_ixgb.py

In [28]:
%cd ~/ml_workshop/onedal

/home/u58475/ml_workshop/onedal


In [ ]:
#choose one of the datasets below

DATASET = letters_data
#DATASET = abalone_data

if DATASET = letters_data:
    !wget https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data --quiet -O data 
else
    !wget https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data --quiet -O data

In [22]:
%%writefile benchmark_ixgb.py
import xgboost as xgb
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np 
import time

df = pd.read_csv('data',header=None)

y,X = df.iloc[:,0],df.iloc[:,1:] # colon represents all the rows, 0 represent first column . # iloc index by number, loc is by name (but in this case no diff bcos index is by number)
enc = LabelEncoder() 
enc.fit(y)
len(enc.classes_) # encodes A -> 0, B -> 1, C -> 2 ... Z -> 25

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train_encoded=enc.transform(y_train) # encoded is converted from letter to an array . won't see index because encode force is to become numpy array. but doesn't matter because we have position (fixed in array)
y_test_encoded=enc.transform(y_test)
y_train_encoded 

dtrain = xgb.DMatrix(X_train, label=y_train_encoded) # class is a template to create objects. every object in the class has same format
dtest = xgb.DMatrix(X_test)

xgb_params = {'alpha': 0.9, 'max_bin': 256, 'scale_pos_weight': 2, 'learning_rate': 0.1, 'subsample': 1, 'reg_lambda': 1, "min_child_weight": 0, 
              'max_depth': 8, 'max_leaves': 2**8, 'tree_method': 'hist', 'predictor': 'cpu_predictor','objective': 'multi:softmax','num_class': len(enc.classes_)} 

t1 = time.time()
model_xgb = xgb.train(xgb_params, dtrain,1000,verbose_eval = 5000)
t2 = time.time()

print("trg duration: ", t2 - t1)
result_predict_xgb_train = model_xgb.predict(dtrain)
train_acc = np.mean(y_train_encoded ==result_predict_xgb_train) # == generates array of 1 and 0. then will mean this 1 and 0 array to get accuracy
print("train_acc: ", train_acc)
result_predict_xgb_test = model_xgb.predict(dtest)
test_acc = np.mean(y_test_encoded==result_predict_xgb_test)
print("test_acc: ", test_acc)

Overwriting benchmark_ixgb.py


In [31]:
!cat benchmark_ixgb.py

import xgboost as xgb
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np 
import time
import os
os.environ["MKL_VERBOSE"] ='1'

df = pd.read_csv('letter_data',header=None)

y,X = df.iloc[:,0],df.iloc[:,1:] # colon represents all the rows, 0 represent first column . # iloc index by number, loc is by name (but in this case no diff bcos index is by number)
enc = LabelEncoder() 
enc.fit(y)
len(enc.classes_) # encodes A -> 0, B -> 1, C -> 2 ... Z -> 25

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train_encoded=enc.transform(y_train) # encoded is converted from letter to an array . won't see index because encode force is to become numpy array. but doesn't matter because we have position (fixed in array)
y_test_encoded=enc.transform(y_test)
y_train_encoded 

dtrain = xgb.DMatrix(X_train, label=y_train_encoded) # class is a template to create objects. every ob

In [13]:
%%writefile submit_ixgb.sh
lscpu
# activate intel xgb environment
source ~/ml_workshop/onedal/bin/activate
#check xgb version
python -c "import xgboost; print('xgboost ',xgboost.__version__, 'installed successfully')"
# run training benchmarks
python ~/ml_workshop/onedal/benchmark_ixgb.py
#deactivate environment
deactivate

Overwriting submit_ixgb.sh


### Check content of benchmark_ixgb.sh

In [33]:
!cat submit_ixgb.sh

lscpu
# activate intel xgb environment
source ~/onedal/bin/activate
#check xgb version
python -c "import xgboost; print('xgboost ',xgboost.__version__, 'installed successfully')"
# run training benchmarks
python ~/benchmark_ixgb.py
#deactivate environment
deactivate


### Optional step : Remove all old output files

In [ ]:
!rm -rf submit_ixgb.sh.*

In [8]:
!rm -rf benchmark_ixgb.sh.*

### Submit to queue. 

In [23]:
!qsub submit_ixgb.sh -l nodes=1:ppn=2 -d .

783756.v-qsvr-1.aidevcloud


### check Job status

In [26]:
!qstat

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
783597.v-qsvr-1            ...ub-singleuser u58475          00:01:08 R jupyterhub     


### If the job status is finished, check the the output file with proper output name.

In [19]:
!cat submit_ixgb.sh.o*


########################################################################
#      Date:           Fri Jan 22 00:36:03 PST 2021
#    Job ID:           783748.v-qsvr-1.aidevcloud
#      User:           u58475
# Resources:           neednodes=1:ppn=2,nodes=1:ppn=2,walltime=06:00:00
########################################################################

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              24
On-line CPU(s) list: 0-23
Thread(s) per core:  2
Core(s) per socket:  6
Socket(s):           2
NUMA node(s):        2
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) Gold 6128 CPU @ 3.40GHz
Stepping:            4
CPU MHz:             1200.697
CPU max MHz:         3700.0000
CPU min MHz:         1200.0000
BogoMIPS:            6800.00
Virtualization:      VT-x
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:       

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Notedown the number of images/sec for training and inference

In [1]:
!grep -e "trg duration" -e "train_acc" -e "test_acc"  submit_ixgb.sh.o*

grep: submit_ixgb.sh.o*: No such file or directory


### close the notebook